In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from collections import defaultdict

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv('../data/processed/train-normalized.csv.gz', skipinitialspace = True)
data

,ncodpers,sexo,age,lat,lon,fecha_alta,ind_nuevo,antiguedad,indrel,ind_empleado_A,ind_empleado_B,ind_empleado_F,ind_empleado_N,ind_empleado_S,ult_fec_cli_1t,indrel_1mes,tiprel_1mes_A,tiprel_1mes_I,tiprel_1mes_P,tiprel_1mes_R,indresi,indext,indfall,cod_prov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,15889,1.0,0.333333,0.764151,0.355072,0.000000,0.0,0.999999,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.011287,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,15890,1.0,0.376543,0.764151,0.355072,0.000000,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.002473,0.333333,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0
2,15891,0.0,0.351852,0.764151,0.355072,0.960543,0.0,0.999990,0.0,0.0,0.0,0.0,1.0,0.0,0.982361,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,0.0,0.000000,0.666667,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,15892,0.0,0.370370,0.764151,0.355072,0.000000,0.0,1.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.014898,0.333333,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
4,15893,1.0,0.376543,0.764151,0.355072,0.126954,0.0,1.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.014898,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,15894,1.0,0.358025,0.764151,0.355072,0.000000,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.009751,0.333333,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
6,15895,0.0,0.296296,0.764151,0.355072,0.000000,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.007110,0.666667,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
7,15896,1.0,0.296296,0.764151,0.355072,0.000000,0.0,1.000000,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.006334,0.333333,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,15897,1.0,0.370370,0.764151,0.355072,0.000000,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.538462,1.0,0.006072,0.333333,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0
9,15898,0.0,0.364198,0.764151,0.355072,0.235204,0.0,1.000000,1.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.538462,0.0,0.006072,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
ids = data['ncodpers']
data.drop(['ncodpers'], axis=1, inplace=True)

In [4]:
models = {}
model_preds = {}
id_preds = defaultdict(list)
products = ['ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 'ind_cco_fin_ult1',
            'ind_cder_fin_ult1', 'ind_cno_fin_ult1', 'ind_ctju_fin_ult1',
            'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 'ind_ctpp_fin_ult1',
            'ind_deco_fin_ult1', 'ind_deme_fin_ult1', 'ind_dela_fin_ult1',
            'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 'ind_hip_fin_ult1',
            'ind_plan_fin_ult1', 'ind_pres_fin_ult1', 'ind_reca_fin_ult1',
            'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 'ind_viv_fin_ult1',
            'ind_nomina_ult1', 'ind_nom_pens_ult1', 'ind_recibo_ult1']

In [5]:
for product in products:
        print(product)
        y_train = data[product]
        x_train = data.drop(product, 1)
        
        clf = LogisticRegression()
        clf.fit(x_train, y_train)
        p_train = clf.predict_proba(x_train)[:,1]
        
        models[product] = clf
        model_preds[product] = p_train
        for id, p in zip(ids, p_train):
            id_preds[id].append(p)
            
        print(roc_auc_score(y_train, p_train))

ind_ahor_fin_ult1
0.795155849295
ind_aval_fin_ult1
0.985470876599
ind_cco_fin_ult1
0.818665613379
ind_cder_fin_ult1
0.913463429803
ind_cno_fin_ult1
0.982061466995
ind_ctju_fin_ult1
0.999322802354
ind_ctma_fin_ult1
0.89420617015
ind_ctop_fin_ult1
0.90905932369
ind_ctpp_fin_ult1
0.85399602339
ind_deco_fin_ult1
0.896268244102
ind_deme_fin_ult1
0.924823934988
ind_dela_fin_ult1
0.928873279844
ind_ecue_fin_ult1
0.89667445137
ind_fond_fin_ult1
0.935840900175
ind_hip_fin_ult1
0.963753208797
ind_plan_fin_ult1
0.933606005783
ind_pres_fin_ult1
0.92345586677
ind_reca_fin_ult1
0.905082744965
ind_tjcr_fin_ult1
0.93998362604
ind_valo_fin_ult1
0.923484409768
ind_viv_fin_ult1
0.908881021468
ind_nomina_ult1
0.999243071185
ind_nom_pens_ult1
0.999326929078
ind_recibo_ult1
0.926853588909


In [9]:
already_active = defaultdict(list)

for i in range(len(data)):  
    id_ = ids.at[i]
    active_products = []
    
    for p in products:
        if data.at[i, p] == 1.0:
            active_products.append(p)
    
    already_active[id_] = active_products

In [10]:
test = pd.read_csv('../data/raw/test.csv.zip', skipinitialspace = True, usecols=['ncodpers'])

In [18]:
most_popular = ['ind_cco_fin_ult1', 'ind_recibo_ult1', 'ind_ctop_fin_ult1', 'ind_cno_fin_ult1', 'ind_ecue_fin_ult1', 'ind_nom_pens_ult1', 'ind_nomina_ult1']

train_preds = {}

for id_ in test.ncodpers.values:
    preds_probas = id_preds[id_]
    active = already_active[id_]    
    predicted = []
    
    for prod_proba_idx in np.argsort(preds_probas)[::-1]:
        if products[prod_proba_idx] not in active and len(predicted) < 7:
            predicted.append(products[prod_proba_idx])
    
    train_preds[id_] = most_popular if len(predicted) == 0 else predicted

In [19]:
for ncodper in test.ncodpers.values:        
    train_preds[ncodper] = ' '.join(train_preds[ncodper])

In [20]:
predictions = pd.DataFrame(list(train_preds.items()), columns=['ncodpers', 'added_products'])
predictions.head()

,ncodpers,added_products
0,15889,ind_recibo_ult1 ind_ctop_fin_ult1 ind_plan_fin...
1,15890,ind_valo_fin_ult1 ind_reca_fin_ult1 ind_dela_f...
2,15892,ind_ctpp_fin_ult1 ind_ctop_fin_ult1 ind_plan_f...
3,15893,ind_cco_fin_ult1 ind_ctop_fin_ult1 ind_ecue_fi...
4,15894,ind_ctpp_fin_ult1 ind_fond_fin_ult1 ind_ctop_f...


In [21]:
predictions.to_csv('../data/predicted.csv.gz', index=False, compression='gzip')